# How to enable versioning Amazon S3
> Boto3 Style 🐍

### 2. Create an S3 bucket.

>> Create bucket function

In [1]:
import logging
import boto3
from botocore.exceptions import ClientError
import os
import json
def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region

    If a region is not specified, the bucket is created in the S3 default
    region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-west-2'
    :return: True if bucket created, else False
    """

    # Create bucket
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [3]:
bucket_name='whizlabs-23456'
s3Client = boto3.client('s3')
create_bucket(bucket_name=bucket_name)
response = s3Client.list_buckets()
names=[]

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    names.append(bucket['Name'])
    print(f'  {bucket["Name"]}')

Existing buckets:
  whizlabs-23456


### 3. Enable versioning on the S3 bucket.

In [10]:
s3Res=boto3.resource('s3')
versioning = s3Res.BucketVersioning(bucket_name)

#check versioning status
print(versioning.status)

# enable versioning
versioning.enable()

#check versioning status
print(versioning.status)

None
Enabled


### 4. Upload an object and make the bucket public.

In [11]:
def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [ ]:
filename='sample.txt'
with open(filename, "rb") as f:
    s3Client.upload_fileobj(f, bucket_name,filename)

s3Client.list_objects(Bucket=bucket_name)

In [ ]:
# Create the bucket policy
bucket_policy = {
   "Id": "Policy1",
   "Version": "2012-10-17",
   "Statement": [
      {
         "Sid": "Stmt1",
         "Action": [
            "s3:GetObject"
         ],
         "Effect": "Allow",
        'Resource': "arn:aws:s3:::%s/*" % bucket_name,       
        "Principal": "*"
      }
   ]
}
# Convert the policy to a JSON string
bucket_policy = json.dumps(bucket_policy)

# Set the new policy on the given bucket
s3Client.put_bucket_policy(Bucket=bucket_name, Policy=bucket_policy)

### 5. Upload different versions of the file.
> Change the version number of the Sample.txt file

In [ ]:

with open(filename, "rb") as f:
    s3Client.upload_fileobj(f,bucket_name,filename)

s3Client.list_objects(Bucket=bucket_name)